In [44]:
from random import randint
from copy import deepcopy

class Elevator(object):
    """An elevator class. It has position, passengers inside and direction. 
    """
    # should we add something for max capacity?

    def __init__(self, n_floors, n_passengers,building,capacity = 15):
        """missing"""
        self.num_of_floors = n_floors
        self.current_floor = 0
        self.direction = True # Would be nice to use boolean instead
        self.register_list = []
        self.step_list = []
        self.capacity = capacity
        self.strategy = 0 
        self.passengers_num = n_passengers
        self.building = building # call building 

 
        self.waiting_list = []
        for i in range(self.passengers_num):
            self.waiting_list.append(Passenger(i, self.num_of_floors))
            self.waiting_list = sorted(self.waiting_list, key=lambda x: x.start_floor)           
                 #sorted for performance reason

    def move(self):
        """Method for elevator movement.
        In every step elevator either goes a floor higher or a floor lower.
        """
        self.current_floor += 1 if self.direction else -1
    
    def enter_passengers(self):
        """Method to get in all passengers and erase them from the list
        """
        for passenger in self.waiting_list:
            if passenger.start_floor == self.current_floor and len(self.register_list) < self.capacity:
                self.register_list.append(passenger)
                self.waiting_list.remove(passenger)
                print ("Elevator picks up passenger %d on floor %d"% (passenger.ID, passenger.start_floor))
            else:
                self.step_list.append(passenger)
                #print ("length of step list",len(self.step_list))

            
    def exit_passengers(self):
        """Function to remove all passengers on their destination floor.
        It is called in run method of the building class.
        """
        for passenger in self.register_list:
            if passenger.dst_floor == self.current_floor:
                self.register_list.remove(passenger)
                self.building.list_of_waittimes.append(self.building.time_step - passenger.start_waittime)
                passenger.end_waittime = self.building.time_step
                #self.end_waittime = self.building.time_step - self.start_waittime
                print("Elevator drops off passenger %d on floor %d"% (passenger.ID, passenger.dst_floor))
            #else:
                #self.building.time_step += 1 
            #print ("time steps",passenger.start_waittime)
            
            
    def direction_default_strategy(self):
        """Default function of elewator work - 
        it starts with going to the roof of building, then
        comes back to the first floor.
        """
        if self.current_floor >= self.num_of_floors - 1:
            self.direction = False
        elif self.current_floor <= 0:
            self.direction = True

    def direction_bad_strategy(self):
        """Example function of very bad strategy. Elevator takes desired floor of the first
        passenger and elevates him to desired location, then repeats for the next passenger.
        If no passenger is in elevator it just goes to the up and down, as in default strategy.
        """
        if len(self.register_list) is 0:
            self.direction_default_strategy()
            return
        firstval = self.register_list[0].dst_floor
        if self.current_floor > firstval:
            self.direction = False
        else:
            self.direction = True
        #print ("bad!")

In [45]:
class Passenger(object):
    """A passenger class. 
    A passenger has his or her ID number, starting floor and destination floor number
    """

    def __init__(self, ID, floorsNum):
        """Initializes values for passenger and select random floors to start
        Destination floor and starting floor differ
        """
        self.ID = ID
        self.start_floor = randint(0, floorsNum-1)  #works for now, would be interesting to use different distributions
        self.dst_floor = randint(0, floorsNum-2)    #same as above
        self.start_waittime = 0
        self.end_waittime = None # changed 0 to None 

        if self.dst_floor == self.start_floor:
            self.dst_floor += 1 # this will bias the destination floors upward

In [46]:
class Building(object):
    """A building class.
    Building had number of floors, list of passengers outside the elevator and its own elevator object.
    Strategy of an elevator is set by integer, where 0 is default and 1 is another very naive strategy.
    """
    MOVE_COST_FACTOR = 2
    ENTER_COST_FACTOR = 1
    EXIT_COST_FACTOR = 1 


    def __init__(self):
        """Makes building class and addes lust of passengers
        The list is sorted by starting floor value for efficacy.
        """
        self.num_of_floors = self.get_value("Please write number of floors in the building: ",\
                                   "Incorrect value. Number of floors should be integer higher than 1.", 2)

        self.passengers_num = self.get_value("Please write number of passengers: ",\
                                    "Incorrect value. Number of passengers should be non-negative integer.", 0)
        self.elevator = Elevator(self.num_of_floors,self.passengers_num,self)
        self.time_step = 0 
        self.list_of_waittimes = [] 
    
    def get_value(self,message,incorret_message,minimal):
        """Method for making sure to obtain integers
        """
        val = None
        try:
            val = int(raw_input(message))
        except ValueError:
            print incorret_message
            return self.get_value(message, incorret_message, minimal)
        if val < minimal:
            print incorret_message
            return self.get_value(message, incorret_message, minimal)
        else:
            return val

    ### Needs to move into elevator
    ### let's consider capacity

    def run(self):
        """Step function. When called:
        1. waiting passengers enter the elevator (register_passenger)
        2. the elevator is assigned a direction value
        3. elevator moves one floor up or down
        4. passengers on destination floors leaves the elevator (cancel_passenger)
        """
        self.elevator.enter_passengers()
        if self.elevator.strategy == 0:
            self.elevator.direction_default_strategy()
        else:
            self.elevator.direction_bad_strategy()
        self.elevator.move()
        self.elevator.exit_passengers()
        self.time_step += Building.MOVE_COST_FACTOR * 1  + Building.EXIT_COST_FACTOR * 1 + Building.ENTER_COST_FACTOR * 1 

    def output(self):
        """Return total number of steps taken
        """
        total_runs = 0 
        while self.elevator.waiting_list or self.elevator.register_list:
            self.run()
            total_runs += 1
        return total_runs
    
    def waittime(self):
        """Return wait time of passengers
        """
        for passenger in self.elevator.step_list:
            passenger.end_waittime = self.time_step - passenger.start_waittime
            return (passenger.end_waittime)
        
    def testlist(self):
        """Test if elevator step list is working
        """
        while self.elevator.waiting_list or self.elevator.register_list:
            return len(self.elevator.register_list)
            #for passenger in self.elevator.step_list:
                #print (passenger.start_waittime)

In [49]:
 def main():
    """Main function"""
    building_def = Building()
    building_bad = deepcopy(building_def)
    print ("Number of steps for default strategy:", building_def.output())
    #print("testlist",building_def.testlist())
    print ("Wait time of passengers for default strategy",building_def.list_of_waittimes)
    building_bad.elevator.strategy = 1
    print ("Number of steps for bad strategy:", building_bad.output())
    print ("Wait time of passengers for bad strategy",building_bad.list_of_waittimes)
if __name__ == "__main__":
    main()

Please write number of floors in the building: 10
Please write number of passengers: 10
Elevator picks up passenger 2 on floor 4
Elevator picks up passenger 9 on floor 5
Elevator picks up passenger 4 on floor 6
Elevator drops off passenger 2 on floor 8
Elevator picks up passenger 0 on floor 8
Elevator picks up passenger 3 on floor 8
Elevator picks up passenger 6 on floor 8
Elevator drops off passenger 9 on floor 8
Elevator picks up passenger 1 on floor 8
Elevator picks up passenger 8 on floor 8
Elevator picks up passenger 7 on floor 4
Elevator drops off passenger 0 on floor 3
Elevator drops off passenger 3 on floor 2
Elevator drops off passenger 1 on floor 2
Elevator drops off passenger 7 on floor 2
Elevator drops off passenger 4 on floor 1
Elevator drops off passenger 6 on floor 1
Elevator drops off passenger 8 on floor 2
Elevator picks up passenger 5 on floor 8
Elevator drops off passenger 5 on floor 7
('Number of steps for default strategy:', 29)
('Wait time of passengers for defaul

start time of passengers stays zero, end time gets updated when they exit to the current building time.